In [26]:
import pandas as pd
import numpy as np
import os
from skimage import transform
import matplotlib.pyplot as plt

#%%Resizeing Images from (1024,1024) to (224,224)

def replaceMultiple(mainString, toBeReplaces, newString): #replacement of multiple characters to use in the createPostName function
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString 


def preprocess_data(n_to_process=-1, img_shape=(224,224)):

	os.makedirs(f"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/database/database_preprocessed/" , exist_ok=True)

	train_data = pd.read_csv(r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/dataset/train_1.txt", header=None, index_col=None)[0].str.split(' ', 1)
	val_data   = pd.read_csv(r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/dataset/val_1.txt", header=None, index_col=None)[0].str.split(' ', 1)
	test_data  = pd.read_csv(r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/dataset/test_1.txt", header=None, index_col=None)[0].str.split(' ', 1)

	# number of samples to process
	train_data = train_data if (n_to_process==-1 or n_to_process>len(train_data)) else train_data[:n_to_process]
	val_data   = val_data if (n_to_process ==-1 or n_to_process>len(val_data)) else val_data[:n_to_process]
	test_data  = test_data if (n_to_process ==-1 or n_to_process>len(test_data)) else test_data[:n_to_process]


	train_paths = train_data.apply(lambda x: r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/database/" + x[0]).to_numpy()
	val_paths   = val_data.apply(lambda x: r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/database/" + x[0]).to_numpy()
	test_paths  = test_data.apply(lambda x: r"C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/database/"+ x[0]).to_numpy()
	all_paths   = np.hstack((train_paths, val_paths, test_paths))

	i=0
	for img_path in all_paths:
		i += 1
		if  i % max(1, int(len(all_paths)/1000))==0: print(i, '/', len(all_paths))
		new_path = img_path.replace('database', 'database_preprocessed'); new_path = replaceMultiple(new_path,['images_001/', 'images_002/', 'images_003/','images_004/', 'images_005/','images_006/', 'images_007/','images_008/', 'images_009/', 'images_010/', 'images_011/', 'images_012/'],'')
		img = plt.imread(img_path)
		img = transform.resize(img, img_shape, anti_aliasing=True)
		plt.imsave(fname=new_path, arr=img, cmap='gray')

preprocess_data(n_to_process=-1, img_shape=(224,224))

112 / 112120
224 / 112120
336 / 112120
448 / 112120
560 / 112120
672 / 112120
784 / 112120
896 / 112120
1008 / 112120
1120 / 112120
1232 / 112120
1344 / 112120
1456 / 112120
1568 / 112120
1680 / 112120
1792 / 112120
1904 / 112120
2016 / 112120
2128 / 112120
2240 / 112120
2352 / 112120
2464 / 112120
2576 / 112120
2688 / 112120
2800 / 112120
2912 / 112120
3024 / 112120
3136 / 112120
3248 / 112120
3360 / 112120
3472 / 112120
3584 / 112120
3696 / 112120
3808 / 112120
3920 / 112120
4032 / 112120
4144 / 112120
4256 / 112120
4368 / 112120
4480 / 112120
4592 / 112120
4704 / 112120
4816 / 112120
4928 / 112120
5040 / 112120
5152 / 112120
5264 / 112120
5376 / 112120
5488 / 112120
5600 / 112120
5712 / 112120
5824 / 112120
5936 / 112120
6048 / 112120
6160 / 112120
6272 / 112120
6384 / 112120
6496 / 112120
6608 / 112120
6720 / 112120
6832 / 112120
6944 / 112120
7056 / 112120
7168 / 112120
7280 / 112120
7392 / 112120
7504 / 112120
7616 / 112120
7728 / 112120
7840 / 112120
7952 / 112120
8064 / 112120


In [23]:
len(all_paths)

112120

In [164]:
import numpy as np
import pandas as pd
import cv2
import os
from glob import glob
from itertools import chain
from numpy import save
#%% Data Selection

all_xray_df = pd.read_csv(r'C:\Users\siris\OneDrive\Desktop\215\Final_Project\data\COVID-CAPS-master\Data_Entry_2017.csv') #loading the csv file
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('.', 'data', 'images*', '*.png'))}


all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]

#Converting Labels to 0 and 1
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)


print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(all_xray_df[c_label].sum())) for c_label in all_labels])


X = all_xray_df.iloc[:,:].values # Converting datafram to numpy array

Y = X[:,12::] #Selecting Labels as Y
X_paths = X[:,0] #File names

Y_cat = np.zeros((Y.shape[0],5)) #Empty array for new categories

#Creating new categories
Y_cat[:,0] = Y[:,10] #No finding
Y_cat[:,1] = Y[:,8] + Y[:,9] + Y[:,11] #Tumor
Y_cat[:,2] = Y[:,4] + Y[:,12] + Y[:,14] #Pleural disease
Y_cat[:,3] = Y[:,2] + Y[:,13] #Lung infection
Y_cat[:,4] = Y[:,0] + Y[:,1] + Y[:,3] + Y[:,5] + Y[:,6] + Y[:,7] #Others

Y_cat = (Y_cat>=1)*1

#Removing cases with multiple labels
cat_sum = np.sum(Y_cat,axis=1)
nonselect_id = []
for i in range(len(cat_sum)):
    if cat_sum[i]>1:
        nonselect_id.append(i) 

#Reduced dataset without multi-labels        
X_r = np.delete(X,nonselect_id,0)
Y_cat_r =  np.delete(Y_cat,nonselect_id,0) #final labels
X_paths_r = X_r[:,0] #final paths



#%%Converting Train Images/Labels into numpy array
# Note : Running this section may take several hours
data_path = r'C:/Users/siris/OneDrive/Desktop/215/Final_Project/data/COVID-CAPS-master/database_preprocessed/'
m=0
X_image = cv2.imread(data_path + X_paths_r[92500])
X_image = np.expand_dims(X_image, axis = 0)
Y_labels = Y_cat_r[92501]
Y_labels = np.expand_dims(Y_labels, axis = 0)
k = 0 #counter
checkpoint=0
p=1
for i in range(92501,94323):
    if checkpoint==1:
        p=p+1
        checkpoint=0
    next_image = cv2.imread(data_path + X_paths_r[i])
    next_image = np.expand_dims(next_image, axis = 0)
    next_label =  Y_cat_r[i]
    next_label = np.expand_dims(next_label, axis = 0)
    X_image = np.concatenate((X_image,next_image), axis=0)
    Y_labels = np.concatenate((Y_labels,next_label), axis=0)
    
    k+=1 #Counter to check the progress
   
    if (k%500==0):
        print(k,'/',len(X_paths_r))
    if (k%1822==0):
        print(k,'/',len(X_paths_r))
        m=m+1
        if m>0:
          p=p+1
          save(r'C:\Users\siris\OneDrive\Desktop\215\Final_Project\data\COVID-CAPS-master\numpy\X_image{}'.format(38) ,X_image)
          save(r'C:\Users\siris\OneDrive\Desktop\215\Final_Project\data\COVID-CAPS-master\numpy\Y_labels{}'.format(38),Y_labels)
          print('saved')
          
   

Clean Labels (15) [('Atelectasis', 11559), ('Cardiomegaly', 2776), ('Consolidation', 4667), ('Edema', 2303), ('Effusion', 13317), ('Emphysema', 2516), ('Fibrosis', 1686), ('Hernia', 227), ('Infiltration', 19894), ('Mass', 5782), ('No Finding', 60361), ('Nodule', 6331), ('Pleural_Thickening', 3385), ('Pneumonia', 1431), ('Pneumothorax', 5302)]
500 / 94323
1000 / 94323
1500 / 94323
1822 / 94323
saved


#  # this for loop was used to create numpy arrays of the 94323 images each of size 2500. Here only the last bit 92500 to 94323 is shown.  

In [147]:

x_train=np.load(r'C:\Users\siris\OneDrive\Desktop\215\Final_Project\data\COVID-CAPS-master\numpy\X_image1.npy') 


In [165]:
X_image.shape

(1823, 224, 224, 3)